In [ ]:
# Predict the results of football matches of the Spanish First Division League
# Eduardo Sthory

In [ ]:
# Libraries
# import warnings filter
import warnings
from sklearn.exceptions import DataConversionWarning
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)
simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

import numpy as np
import pandas as pd
from IPython.display import display
from Functions import result_to_numeric, best_features
#from Functions.ipynb import result_to_numeric, best_features
import Functions 
from Features_Engineer import features_engineer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import scale
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sea

From: http://football-data.co.uk
Name of the results dataset fields SOURCE:

    - SP1-2009.csv
    - SP1-2010.csv
    - SP1-2011.csv
    - SP1-2012.csv
    - SP1-2013.csv   
    - SP1-2014.csv
    - SP1-2015.csv
    - SP1-2016.csv
    - SP1-2017.csv
    - SP1-2018.csv
 
 Fields:
 
 - Div = League Division
 - Date = Match Date (dd/mm/yy)
 - HomeTeam = Home Team
 - AwayTeam = Away Team
 - FTHG and HG = Full Time Home Team Goals
 - FTAG and AG = Full Time Away Team Goals
 - FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
 - HTHG = Half Time Home Team Goals
 - HTAG = Half Time Away Team Goals
 - HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
 
 Match Statistics 
 
 - HS = Home Team Shots
 - AS = Away Team Shots
 - HST = Home Team Shots on Target
 - AST = Away Team Shots on Target
 - HC = Home Team Corners
 - AC = Away Team Corners
 - HF = Home Team Fouls Committed
 - AF = Away Team Fouls Committed

In [ ]:
# Load 10 last Season 2009-2018
data1 = pd.read_csv('SP1-2009.csv')
data2 = pd.read_csv('SP1-2010.csv')
data3 = pd.read_csv('SP1-2011.csv')
data4 = pd.read_csv('SP1-2012.csv')
data5 = pd.read_csv('SP1-2013.csv')
data6 = pd.read_csv('SP1-2014.csv')
data7 = pd.read_csv('SP1-2015.csv')
data8 = pd.read_csv('SP1-2016.csv')
data9 = pd.read_csv('SP1-2017.csv')
data10 = pd.read_csv('SP1-2018.csv')

# Eliminate unnecessary columns
data1 = data1.iloc[:,:18]
data2 = data2.iloc[:,:18]
data3 = data3.iloc[:,:18]
data4 = data4.iloc[:,:18]
data5 = data5.iloc[:,:18]
data6 = data6.iloc[:,:18]
data7 = data7.iloc[:,:18]
data8 = data8.iloc[:,:18]
data9 = data9.iloc[:,:18]
data10 = data10.iloc[:,:18]

# Drop Div, Date and others
data1 = data1.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data2 = data2.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data3 = data3.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data4 = data4.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data5 = data5.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data6 = data6.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data7 = data7.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data8 = data8.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data9 = data9.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)
data10 = data10.drop(['Div','Date','HTHG','HTAG','HTR'],axis=1)

# Transform FTR (Target)
data1['FTR'] = data1.apply(lambda row: result_to_numeric(row),axis=1)
data2['FTR'] = data2.apply(lambda row: result_to_numeric(row),axis=1)
data3['FTR'] = data3.apply(lambda row: result_to_numeric(row),axis=1)
data4['FTR'] = data4.apply(lambda row: result_to_numeric(row),axis=1)
data5['FTR'] = data5.apply(lambda row: result_to_numeric(row),axis=1)
data6['FTR'] = data6.apply(lambda row: result_to_numeric(row),axis=1)
data7['FTR'] = data7.apply(lambda row: result_to_numeric(row),axis=1)
data8['FTR'] = data8.apply(lambda row: result_to_numeric(row),axis=1)
data9['FTR'] = data9.apply(lambda row: result_to_numeric(row),axis=1)
data10['FTR'] = data10.apply(lambda row: result_to_numeric(row),axis=1)

In [ ]:
# Union data
# It was decided to work with the last 4 seasons
features = pd.concat([data1,data2,data3,data4,data5,data6,
                      data7, data8, data9, data10], ignore_index=True)
target = pd.concat([data1['FTR'],data2['FTR'],data3['FTR'],
                    data4['FTR'],data5['FTR'],data6['FTR'], 
                    data7['FTR'],data8['FTR'],data9['FTR'], 
                    data10['FTR']],ignore_index=True)

In [ ]:
# Get features fields more importants
features_tmp = features[['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC']]
# Rank Best features
tmp=best_features(features_tmp, target)
print(tmp)  

The FTAG, FTHG, AST and HST characteristics are the most important, we are going to work with them

# Visualize the best features

In [ ]:
# Barplot feature most important
tmp=Functions.visualize_best_features(tmp)

In [ ]:
# Explore data
from pandas.plotting import scatter_matrix

spm1 = scatter_matrix(features[['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC']],
                       alpha=0.2,figsize=(6,6),diagonal = 'kde')

In [ ]:
corr = features[['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC']].corr()
corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

In [ ]:
features.describe()

In [ ]:
#from scipy.stats import describe
#describe(features[['FTHG','FTAG','HS','AS','HST','AST','HF','AF','HC','AC']], axis=0)

import pandas_profiling as pp
eda = pp.ProfileReport(features)
display(eda)


Drop characteristics with less importance

In [ ]:
# Drop HC, AC, AS, HS, HF, AF
features = features.drop(['HC','AC','AS','HS','HF','AF'],axis=1)

In [ ]:
features.head()

# Features Engineering

Taking into account the above, we will take to work the FTAG, FTHG, AST and HST features.
These characteristics will not be present when the prediction is made, therefore we will 
create new characteristics based on these and that can accumulate in some way the values 
of the training.
These new features will be: 

   - TFTG  accumulated - Full Time Home Team Goals
   - TFTA  accumulated - Full Time Away Team Goals

   - GAHT   accumulated - Goals against HomeTeam
   - GAAT   accumulated - Goals against AwayTeam

   - THST   Total - Home Team Shots on Target
   - TAST   Total - Away Team Shots on Target

   - HTP    Home Total Points accumulate
   - ATP    Away Total Points accumulate 

   - HGA    HomeTeam Goal Average
   - AGA    AwayTeam Goal Average

In [ ]:
# Feature Engineering
features = features_engineer(features, target)

The statistics of the first week (the first 19 records) must be 0, 
since each team plays only once and does not have any antecedents to calculate it.

In [ ]:
features.head(30)

In [ ]:
features.tail()

The new features have the expected importance to train the model, 
remember that the statistics of the game know them after its completion, 
therefore you must work with the data prior to the game, 
this way we can now eliminate those characteristics that "No we must use":
These are: 
    FTAG, FTHG, AST, HST

# Split Train and Test
We can not use the sklearn methods to perform the "splits", since we are working with time series and can not choose random data

In [ ]:
# Choose the characteristics for learning and testing
cols = ['TFTHG','TFTAG', 'GAHT', 'GAAT','HGA','AGA','HTP','ATP',
        'diffgoals','diffshots','diffp','diffg']

# Amount of the test 10%, 90% Train
n = 400     # record for the test

# Split feature_table: Training (only the matches already played)
X_train = features[cols]
X_train = X_train[:-n] # Eliminate the last n rows, (nrows - n) for training

# Split target Train: only the matches already played
y_train = target[:-n] # Eliminate the last n rows, (nrows - n) for training


# Split feature_table: Test (only the games not played)
X_test = features[cols]
X_test = X_test[features.shape[0]-n:] # n rows for the test

# Split target test: only the games not played
y_test = target[target.shape[0]-n:]

# Construction of the models
We will start testing the following models:
    - LogisticRegression
    - xgboost
    - KNeighbors
    - SVC
    - Decision Tree 
    - Random Forest
    - AdaBoost
    - Gradient Boosting
    
For the metrics we will use:
    - f1 Score
    - Cross Val Score
    
For the tuning:
    - GridSearchCV
    
And finally: we will use the "Stacking" technique to try to improve the accuracy

# Training and test models

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

f1_scores = dict()

classifiers = [
    LogisticRegression(random_state=12),
    XGBClassifier(random_state=12),
    KNeighborsClassifier(3),
    SVC(random_state = 42, kernel='rbf', gamma='auto',probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "F1 Score"]
scores = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf_name = clf.__class__.__name__
    
    print("-"*82+"\n")
    print("Clasifier: " + clf_name)
    print('****Results****')

    y_predict_test = clf.fit(X_train, y_train).predict(X_test)
    
    # Accuracy
    acc=accuracy_score(y_test, y_predict_test)
    print("Accuracy Score: " + str(round(100*acc, 2)) + "% \n")

    # F1 Scores
    f1s = f1_score(y_test, y_predict_test, average='weighted')
    print("f1 Score: " + str(round(100 * f1s, 2)) + "% \n")
    
    train_predictions = y_predict_test
    log_entry = pd.DataFrame([[clf_name, acc, f1s ]], columns=log_cols)
    scores = scores.append(log_entry)
    
print("-"*82)

# Visualize the Accuracy Metric

In [ ]:
import seaborn as sns
sns.barplot(x='Accuracy', y='Classifier', data=scores.sort_values(by='Accuracy', ascending = False))
plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')
plt.show()

# Visualize the F1 Score Metric

In [ ]:
sns.barplot(x='F1 Score', y='Classifier', 
            data=scores.sort_values(by='F1 Score', ascending = False))

plt.xlabel('F1 Score')
plt.title('Classifier F1 Score')
plt.show()

# Visualize Accuracy Vs F1 Score

In [ ]:
import matplotlib.patches as mpatches
x = np.arange(len(scores))
bar_width = 0.4
plt.bar(x, scores['Accuracy'], width = bar_width, color = 'orange', zorder=2)
plt.bar(x + bar_width, scores['F1 Score'], width = bar_width, color = 'blue', zorder=2)

plt.xticks(x + bar_width, scores['Classifier'], rotation='vertical')
plt.title('Accuracy Vs F1 Score')
plt.xlabel('Classifiers')
plt.ylabel('Scores')

orange_patch = mpatches.Patch(color='orange', label='Accuracy')
blue_patch = mpatches.Patch(color='blue', label='F1 Score')

plt.legend(handles=[orange_patch,blue_patch])

plt.grid(axis='y')
plt.show()

# The best models (without tuning) are:

Clasifier: SVC
****Results****
Accuracy Score: 50.25% 
f1 Score: 40.56% 

Clasifier: XGBClassifier
****Results****
Accuracy Score: 46.25% 
f1 Score: 40.38%

Clasifier: LogisticRegression
****Results****
Accuracy Score: 47.5% 
f1 Score: 39.99% 

Clasifier: AdaBoostClassifier
****Results****
Accuracy Score: 45.5% 
f1 Score: 39.82% 

# Tuning using GridSearchCV

According to the above, I will take the best classifiers to optimize them:
    - LogisticRegression
    - XGBClassifier
    - SVC
    - AdaBoostClassifier

In [ ]:
# Models for GridSearchCV

models = [{'name': 'Logistic Regression','label': 'Logistic Regression',
           'classifier': LogisticRegression(random_state=49),
           'grid': {'penalty':['l2'],
                    'C':[5,6,7],
                    'solver':['newton-cg','lbfgs','sag','saga'],
                    'max_iter':[100],
                    'multi_class':['ovr','multinomial']}},
          
          {'name': 'Xgboost','label':'Xgboost',
           'classifier': XGBClassifier(random_state=49),
           'grid':{'seed':[0],
                   'n_estimators':[5,10,20],
                   'learning_rate':[0.1],
                   'subsample':[0.8, 0.9],
                   'objective':['binary:logistic'],
                   'max_depth':[2,3,4],
                   'gamma':[1,2,3],
                   'min_child_weight':[2,3,4]}},
          
          {'name': 'SVC (RBF)', 'label': 'SVC (RBF)',
           'classifier': SVC(random_state=49, probability=True ),
           'grid': {'C': [9,10],
                    'gamma': [0.01, 0.001],
                    'kernel': ['linear','rbf','poly','sigmoid']}},
          
          {'name': 'AdaBoost', 'label': 'AdaBoost Classifier',
           'classifier':AdaBoostClassifier(learning_rate=1,
                                           random_state=49),
           'grid': {'n_estimators': [2,4,6],
                    'algorithm':['SAMME','SAMME.R'] }}]


In [ ]:
def model_selection(classifier, name, grid, X_train, y_train, X_test, y_test, scoring1, scoring2):
    
    gridsearch_cv1=GridSearchCV(classifier, 
                               grid,
                               cv=5, 
                               scoring = scoring1)
    
    gridsearch_cv2=GridSearchCV(classifier, 
                               grid,
                               cv=5, 
                               scoring = scoring2)
    
    gridsearch_cv1.fit(X_train, y_train)
    gridsearch_cv2.fit(X_train, y_train)
    
    results_dict = {} # Scores with data training
    
    results_dict['classifier_name'] = name    
    results_dict['classifier'] = gridsearch_cv1.best_estimator_
    results_dict['best_params'] = gridsearch_cv1.best_params_
    
    # Traininig Scores
    results_dict['accuracy-train'] = gridsearch_cv1.best_score_
    results_dict['f1 score-train'] = gridsearch_cv2.best_score_

    # Test Scores    
    y_pred = gridsearch_cv1.best_estimator_.predict(X_test)
    results_dict['accuracy-test'] = accuracy_score(y_test, y_pred, 'accuracy')
    
    y_pred = gridsearch_cv2.best_estimator_.predict(X_test)
    results_dict['f1 score-test'] = f1_score(y_test, y_pred, average='weighted')
    
    return(results_dict)

results = []

for mod in models:    
    print(mod['name'], ".....")    
    results.append(model_selection(mod['classifier'], 
                                   mod['name'],
                                   mod['grid'],
                                   X_train, 
                                   y_train, 
                                   X_test,
                                   y_test,
                                   'accuracy',
                                   'f1_weighted'))      
    print('....ready ', mod['name'])

In [ ]:
results_df = pd.DataFrame(results).sort_values(by='accuracy-test', ascending = False)

In [ ]:
results_df

In [ ]:
results_df[['classifier_name','accuracy-train','accuracy-test','f1 score-train','f1 score-test']]

In [ ]:
xgb_params = results_df[results_df['classifier_name'] == 'Xgboost'].best_params.values
lr_params = results_df[results_df['classifier_name'] == 'Logistic Regression'].best_params.values
svc_params = results_df[results_df['classifier_name'] == 'SVC (RBF)'].best_params.values
ada_params = results_df[results_df['classifier_name'] == 'AdaBoost'].best_params.values

In [ ]:
print("xgb_params: ", xgb_params, "\n\n",
      "lr_params: ", lr_params, "\n\n",
      "svc_params: ", svc_params, "\n\n",
      "ada_params: ", ada_params, "\n")

# Results Tuning

# Best Classifiers after tuning (accuracy-test set):

    - XgBoost, accuracy: 0.4950 
    - SVC, accuracy: 0.4925 
    - Logistic regression, accuracy: 0.4775	 
    - AdaBoostClassifier, accuracy: 0.4750 

# Best Classifiers after tuning (accuracy-training set):

    - XgBoost, accuracy: 0.557059 
    - Logistic regression, accuracy: 0.548529	
    - AdaBoostClassifier, accuracy: 0.540588
    - SVC, accuracy: 0.533529	 

# Stacking
To try to improve the accuracy we will use the technique of "Stacking"

We will use the following models in the layers:
    
    - XgBoost
    - LogisticRegression
    - SVC
    - AdaBoostClassifier
    
And as a last layer (similar to 'softmax') we will use LogisticRegression 

In [ ]:
from mlens.ensemble import SuperLearner
from mlens.ensemble import Subsemble
from mlens.metrics.metrics import rmse

def f1(y, p): return f1_score(y, p, average='weighted')

ensemble = SuperLearner(scorer=f1, random_state=49, verbose=True)

ensemble.add([XGBClassifier(gamma=2, 
                            learning_rate=0.1,
                            max_depth=3,
                            min_child_weight=4,
                            n_estimators=10,
                            objective='binary:logistic',
                            seed=0,
                            subsample=0.9)])

ensemble.add([LogisticRegression(C=6, 
                                 max_iter=100, 
                                 multi_class='ovr', 
                                 penalty='l2', 
                                 solver='lbfgs', 
                                 random_state=49)])
ensemble.add([SVC(C=10, 
                  gamma=0.001, 
                  kernel='rbf',
                  random_state=49)])

ensemble.add([AdaBoostClassifier(algorithm='SAMME.R', 
                                 n_estimators=6,
                                 random_state=49)])


ensemble.add_meta([LogisticRegression(C=6, 
                                 max_iter=100, 
                                 multi_class='ovr', 
                                 penalty='l2', 
                                 solver='lbfgs', 
                                 random_state=49)])

ensemble.fit(X_train, y_train)
preds = ensemble.predict(X_test)
f1(y_test, preds)
accuracy_score(y_test, preds)

In [ ]:
The stacking technique obtained the same accuracy as the best of the models (XgBoost)

# Cross Validation with Times Series

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit

X = features[cols]
y = target

tsv = TimeSeriesSplit(n_splits=3).split(X)

clf = XGBClassifier(gamma=2, 
                    learning_rate=0.1,
                    max_depth=3,
                    min_child_weight=4,
                    n_estimators=10,
                    objective='binary:logistic',
                    seed=0,
                    subsample=0.9)

scores = cross_val_score(clf, X, y, cv=tsv, scoring='accuracy')
    
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


### Conclusion 
### Our selected model has 0.53 and a very small variation of 0.01, which is acceptable
### In this way we match the accuracy of the betting house programs